In [6]:
import pandas as pd
import numpy as np


In [7]:
path1 = r"./prompt_query_pairs.csv"
data = pd.read_csv(path1)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1000 non-null   object
 1   query     1000 non-null   object
 2   prompt    1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


,category,query,prompt
0,nature,Draw a calm beach at dawn,"A tranquil beach with soft waves, palm trees, ..."
1,nature,Can you show me a peaceful forest?,A serene green forest with tall pine trees and...
2,nature,Can you show me a peaceful forest?,A serene green forest with tall pine trees and...
3,nature,I want to see a mountain lake,A crystal clear mountain lake surrounded by sn...
4,nature,What does a desert at sunset look like?,A golden desert with sand dunes glowing orange...


In [16]:
import json 
import os 
import random
import requests
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime
# ! pip install -qU json_repair
import json_repair
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [10]:
raw = data[["query", 'prompt']]

In [20]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct" 

In [25]:
user_prompt = "create an person with red shairt and white car in the day with background in sky add some green trees"

In [26]:
# what i want form the model  
"""

message_structure = {
    "type": "", # type of the scene nature, city, animal, etc...
    "shape": "", #shape of the scene circle, square, etc...
    "objects": [
            {
                "type": "",
                "colors": [],
                "
            }
        ], # objects on the seans
    "colors":[], # color of the objects
    "background":[], # background color of the screen
    "mode": [], # light, night, dark ,day , etc...
}


"""

SceneTypes = Literal["nature", "city", "animal", "animation"]
Shapes = Literal["circle", "square", "triangle", "rectangle"]
ObjectTypes = Literal[
    "human", "man", "woman", "person", "tree", "car", "building", "bridge",
    "mountain", "bird", "fish", "flower", "sun", "moon",
]
Modes = Literal["light", "dark", "day", "night"]

# identfy the object in the scene
class ObjectInScene(BaseModel):
    object_type: ObjectTypes = Field(..., description="Type of the object in the scene")
    colors: List[str] = Field(..., min_items=1, description="List of colors in the object")

# identfy the scene
class Scene(BaseModel):
    scene_type: SceneTypes = Field(..., description="General type of the scene")
    shape: Shapes = Field(..., description="Geometric shape of the scene")
    objects: List[ObjectInScene] = Field(..., min_items=1,max_items=10, description="Objects in the scene")
    colors: List[str] = Field(..., min_items=1, max_items=10,description="Main object colors in the scene")
    background: List[str] = Field(..., min_items=1, max_items=10, description="Background colors of the scene")
    mode: Modes = Field(..., description="Lighting or mood mode like night or day")

In [ ]:
extraction_message = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an expert NLP data parser.",
            "Your task is to extract structured information from the input text.",
            "You must return the extracted data as a valid JSON object based on the Pydantic model provided.",
            "Do not add any explanation, introduction, or summary — just return the raw JSON."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "Prompt:",
            user_prompt.strip(),
            "\nPydantic schema (use this as a reference for keys and types):",
            json.dumps(Scene.model_json_schema(), indent=2, ensure_ascii=False)
        ])
    }
]


In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

